In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

### Scrape Headlines, Link and content

In [66]:
# Define the URL of the website
url = 'https://www.ttnews.com/logistics?page='

page_headings = []

for page in range(2):

    # Send a GET request to the website
    response = requests.get(url+str(page))

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    headlines = soup.find_all(['h2'])

    page_headings.append(headlines)

In [67]:
links=[]
for headlines in page_headings:
    for headline in headlines:
        try:
            links.append(headline.a["href"])
        except:
            links.append(None)

In [68]:
news_headlines = []
for headlines in page_headings:
    for headline in headlines:
        news_headlines.append(headline.text.strip())

In [69]:
df = pd.DataFrame({"Headlines":news_headlines,"URL":links}).dropna()
df["Headlines"] = df["Headlines"].apply(lambda a : None if a.count("Podcasts")!=0 else a)
df.dropna(inplace=True)
df["URL"] = "https://www.ttnews.com"+df["URL"]
df.to_csv("Logistics.csv")
df

,Headlines,URL
1,Update: XPO Spends $870 Million for Yellow Pro...,https://www.ttnews.com/videos/update-xpo-spend...
2,Daily Briefings from Transport Topics,https://www.ttnews.com/listen
3,Breaking: Brad Jacobs' Bold Billion Dollar Move,https://www.ttnews.com/videos/breaking-brad-ja...
4,DHL Express Workers Strike at Cincinnati Air C...,https://www.ttnews.com/articles/dhl-express-st...
5,Dark Horse Bidder Makes Second Offer for Yello...,https://www.ttnews.com/articles/yellow-rejects...
6,Ryder Reveals First Two Companies in EV Fleet ...,https://www.ttnews.com/articles/ryder-two-comp...
7,US-Mexico Trade Trends Indicate Nearshoring Ha...,https://www.ttnews.com/articles/nearshoring-ha...
8,Uber Freight and Torc Robotics Form Partnershi...,https://www.ttnews.com/articles/torc-uber-frei...
9,Vermont Lowers Cost of Milk Hauler Overweight ...,https://www.ttnews.com/articles/vermont-costs-...
10,Recruiting and Retaining Drivers Remains a Cha...,https://www.ttnews.com/articles/retaining-driv...


In [36]:
content = []

for index in tqdm(df.index):

    # Send a GET request to the website
    response = requests.get(df.loc[index,"URL"])

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    paragraphs = soup.find_all(['p'])

    para = df.loc[index,"Headlines"].upper()
    for paragraph in paragraphs:
        para = para + paragraph.text.strip()
    para.replace("\n","|")

    content.append(para)

100%|██████████| 30/30 [00:14<00:00,  2.06it/s]


In [39]:
news = [i.replace('\n', '|') for i in content]

In [40]:
from fpdf import FPDF

news = "\n\n".join(news)

# Create instance of FPDF class
pdf = FPDF()

# Add a page
pdf.add_page()

# Set font
pdf.set_font('Arial', '', 12)

# Split the string into smaller chunks
chunk_size = 500  # Define the size of each chunk
chunks = [news[i:i+chunk_size] for i in range(0, len(news), chunk_size)]

# Add each chunk to the PDF
for chunk in chunks:
    pdf.multi_cell(0, 10, chunk.encode('latin-1', 'replace').decode('latin-1'))
    pdf.ln(10)  # Add a line break after each chunk

# Save the pdf with name .pdf
pdf.output('Government.pdf', 'F')

''

In [26]:
len("\n\n".join(content))

1456505

In [128]:
df["Content"] = content
df['Content'] = df['Content'].str.replace('\n', '|', regex=False)

In [108]:
# Define the URL of the website
url = 'https://www.ttnews.com/government?page='

page_headings = []

for page in tqdm(range(100)):

    # Send a GET request to the website
    response = requests.get(url+str(page))

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    headlines = soup.find_all(['h2'])

    page_headings.append(headlines)

  7%|▋         | 7/100 [00:05<01:11,  1.30it/s]


KeyboardInterrupt: 

In [ ]:
government_headlines = []
for headlines in page_headings:
    for headline in headlines:
        government_headlines.append(headline.text.strip())

In [ ]:
government_headlines = np.where(np.array(government_headlines)!='Top Menu',government_headlines,None)
government_headlines = np.where(np.array(government_headlines)!='Footer menu',government_headlines,None)
government = pd.DataFrame(government_headlines,columns=["Headlines"]).dropna()
government["Headlines"] = government["Headlines"].apply(lambda a : None if a.count("Podcasts")!=0 else a).dropna()
government.to_csv("Technology.csv")

In [ ]:
# Define the URL of the website
url = 'https://www.ttnews.com/technology?page='

page_headings = []

for page in tqdm(range(100)):

    # Send a GET request to the website
    response = requests.get(url+str(page))

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    headlines = soup.find_all(['h2'])

    page_headings.append(headlines)

In [13]:
technology_headlines = []
for headlines in page_headings:
    for headline in headlines:
        technology_headlines.append(headline.text.strip())

In [15]:
technology_headlines = np.where(np.array(technology_headlines)!='Top Menu',technology_headlines,None)
technology_headlines = np.where(np.array(technology_headlines)!='Footer menu',technology_headlines,None)
technology = pd.DataFrame(technology_headlines,columns=["Headlines"]).dropna()
technology["Headlines"] = technology["Headlines"].apply(lambda a : None if a.count("Podcasts")!=0 else a).dropna()
technology.to_csv("Technology.csv")

In [9]:
# Define the URL of the website
url = 'https://www.ttnews.com/safety?page='

page_headings = []

for page in tqdm(range(100)):

    # Send a GET request to the website
    response = requests.get(url+str(page))

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    headlines = soup.find_all(['h2'])

    page_headings.append(headlines)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:42<00:00,  2.36it/s]


In [10]:
safety_headlines = []
for headlines in page_headings:
    for headline in headlines:
        safety_headlines.append(headline.text.strip())

In [14]:
safety_headlines = np.where(np.array(safety_headlines)!='Top Menu',safety_headlines,None)
safety_headlines = np.where(np.array(safety_headlines)!='Footer menu',safety_headlines,None)
safety = pd.DataFrame(safety_headlines,columns=["Headlines"]).dropna()
safety["Headlines"] = safety["Headlines"].apply(lambda a : None if a.count("Podcasts")!=0 else a).dropna()
safety.to_csv("Safety.csv")

In [11]:
# Define the URL of the website
url = 'https://www.ttnews.com/logistics?page='

page_headings = []

for page in tqdm(range(100)):

    # Send a GET request to the website
    response = requests.get(url+str(page))

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    headlines = soup.find_all(['h2'])

    page_headings.append(headlines)

100%|██████████| 100/100 [00:41<00:00,  2.38it/s]


In [3]:
logistics_headlines = []
for headlines in page_headings:
    for headline in headlines:
        logistics_headlines.append(headline.text.strip())

In [4]:
logistics_headlines = np.where(np.array(logistics_headlines)!='Top Menu',logistics_headlines,None)
logistics_headlines = np.where(np.array(logistics_headlines)!='Footer menu',logistics_headlines,None)
logistics = pd.DataFrame(logistics_headlines,columns=["Headlines"]).dropna()
logistics["Headlines"] = logistics["Headlines"].apply(lambda a : None if a.count("Podcasts")!=0 else a).dropna()
logistics.to_csv("Logistics.csv")

In [ ]:
# Define the URL of the website
url = 'https://www.ttnews.com/business?page='

page_headings = []

for page in tqdm(range(100)):

    # Send a GET request to the website
    response = requests.get(url+str(page))

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    headlines = soup.find_all(['h2'])

    page_headings.append(headlines)

In [ ]:
business_headlines = []
for headlines in page_headings:
    for headline in headlines:
        business_headlines.append(headline.text.strip())

In [ ]:
business_headlines = np.where(np.array(business_headlines)!='Top Menu',business_headlines,None)
business_headlines = np.where(np.array(business_headlines)!='Footer menu',business_headlines,None)
business = pd.DataFrame(business_headlines,columns=["Headlines"]).dropna()
business["Headlines"] = business["Headlines"].apply(lambda a : None if a.count("Podcasts")!=0 else a).dropna()
business.to_csv("Business.csv")

In [ ]:
# Define the URL of the website
url = 'https://www.ttnews.com/equipment?page='

page_headings = []

for page in tqdm(range(100)):

    # Send a GET request to the website
    response = requests.get(url+str(page))

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    headlines = soup.find_all(['h2'])

    page_headings.append(headlines)

In [ ]:
equipment_headlines = []
for headlines in page_headings:
    for headline in headlines:
        equipment_headlines.append(headline.text.strip())

In [ ]:
equipment_headlines = np.where(np.array(equipment_headlines)!='Top Menu',equipment_headlines,None)
equipment_headlines = np.where(np.array(equipment_headlines)!='Footer menu',equipment_headlines,None)
equipment = pd.DataFrame(equipment_headlines,columns=["Headlines"]).dropna()
equipment["Headlines"] = equipment["Headlines"].apply(lambda a : None if a.count("Podcasts")!=0 else a).dropna()
equipment.to_csv("Equipment.csv")

In [ ]:
# Define the URL of the website
url = 'https://www.ttnews.com/fuel?page='

page_headings = []

for page in tqdm(range(100)):

    # Send a GET request to the website
    response = requests.get(url+str(page))

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    headlines = soup.find_all(['h2'])

    page_headings.append(headlines)

In [ ]:
fuel_headlines = []
for headlines in page_headings:
    for headline in headlines:
        fuel_headlines.append(headline.text.strip())

In [ ]:
fuel_headlines = np.where(np.array(fuel_headlines)!='Top Menu',fuel_headlines,None)
fuel_headlines = np.where(np.array(fuel_headlines)!='Footer menu',fuel_headlines,None)
fuel = pd.DataFrame(fuel_headlines,columns=["Headlines"]).dropna()
fuel["Headlines"] = fuel["Headlines"].apply(lambda a : None if a.count("Podcasts")!=0 else a).dropna()
fuel.to_csv("Fuel.csv")

In [ ]:
# Define the URL of the website
url = 'https://www.ttnews.com/autonomous?page='

page_headings = []

for page in tqdm(range(100)):

    # Send a GET request to the website
    response = requests.get(url+str(page))

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all headline elements
    # You may need to inspect the website and update the tag and class accordingly
    headlines = soup.find_all(['h2'])

    page_headings.append(headlines)

In [ ]:
autonomous_headlines = []
for headlines in page_headings:
    for headline in headlines:
        autonomous_headlines.append(headline.text.strip())

In [ ]:
autonomous_headlines = np.where(np.array(autonomous_headlines)!='Top Menu',autonomous_headlines,None)
autonomous_headlines = np.where(np.array(autonomous_headlines)!='Footer menu',autonomous_headlines,None)
autonomous = pd.DataFrame(autonomous_headlines,columns=["Headlines"]).dropna()
autonomous["Headlines"] = autonomous["Headlines"].apply(lambda a : None if a.count("Podcasts")!=0 else a).dropna()
autonomous.to_csv("Autonomous.csv")